In [32]:
!pip install pandas

# Data Exploration & Feature Engineering

This notebook processes police press releases and news articles to extract accident datetime information and prepare the data for analysis.

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from typing import Dict, List
import warnings
import re
import os
warnings.filterwarnings('ignore')

# Configuration
DATA_PATH = "C:\\Python\\ics5110-assignment\\data\\"
OUTPUT_PATH = "C:\\Python\\ics5110-assignment\\data\\processed\\"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 1. Load and Explore Datasets

In [34]:
# Load articles dataset
articles_df = pd.read_csv(f"{DATA_PATH}local_news_articles.csv")
print(f"Articles dataset shape: {articles_df.shape}")
articles_df.info()

Articles dataset shape: (321, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   article_id         321 non-null    int64 
 1   url                321 non-null    object
 2   source_name        321 non-null    object
 3   source_url         321 non-null    object
 4   title              321 non-null    object
 5   subtitle           313 non-null    object
 6   author_name        321 non-null    object
 7   publish_date       321 non-null    object
 8   content            321 non-null    object
 9   top_image_url      318 non-null    object
 10  top_image_caption  312 non-null    object
 11  created_at         321 non-null    object
 12  tags               321 non-null    object
 13  categories         321 non-null    object
dtypes: int64(1), object(13)
memory usage: 35.2+ KB


In [35]:
# Load police releases dataset with primary key
police_releases_df = pd.read_csv(f"{DATA_PATH}police_press_releases.csv")
police_releases_df.insert(0, 'release_id', range(1, len(police_releases_df) + 1))

print(f"Police releases dataset shape: {police_releases_df.shape}")
print(f"Primary key range: {police_releases_df['release_id'].min()} to {police_releases_df['release_id'].max()}")
police_releases_df.info()

# Display sample
print("\n" + "="*80)
print("Sample police press release:")
print(f"Release ID: {police_releases_df['release_id'].iloc[0]}")
print(f"Published: {police_releases_df['date_published'].iloc[0]}")
print(f"Content: {police_releases_df['content'].iloc[0][:200]}...")

Police releases dataset shape: (111, 5)
Primary key range: 1 to 111
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   release_id      111 non-null    int64 
 1   title           111 non-null    object
 2   date_published  111 non-null    object
 3   date_modified   111 non-null    object
 4   content         111 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.5+ KB

Sample police press release:
Release ID: 1
Published: 2025-10-09
Content: Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted tha...


# 2. Accident DateTime Extraction Function

Extract accident date and time from police press release content with high accuracy.

In [36]:
def extract_accident_datetime_improved(content: str, published_date: str) -> dict:
    """
    Extract accident date and time from police press release content.
    
    Returns:
        Dictionary with accident_datetime, accident_date, accident_time, time_confidence
    """
    result = {
        'accident_datetime': None,
        'accident_date': None,
        'accident_time': None,
        'time_confidence': 'low'
    }
    
    if pd.isna(content) or pd.isna(published_date):
        return result
    
    content_lower = content.lower()
    published_dt = pd.to_datetime(published_date)
    extracted_time = None
    
    # HIGH confidence: Exact time with "hrs"
    hrs_patterns = [
        (r'at around (\d{4})\s*hrs', 1), (r'at (\d{4})\s*hrs', 1),
        (r'around (\d{4})\s*hrs', 1), (r'\((\d{4})\s*hrs\)', 1),
        (r'\w+\s*\((\d{4})\s*hrs\)', 1),
        (r'at around (\d{1,2}):(\d{2})\s*hrs', 2), (r'at (\d{1,2}):(\d{2})\s*hrs', 2),
        (r'around (\d{1,2}):(\d{2})\s*hrs', 2),
        (r'at around (\d{1,2})\.(\d{2})\s*hrs', 2), (r'at (\d{1,2})\.(\d{2})\s*hrs', 2),
    ]
    
    for pattern, num_groups in hrs_patterns:
        match = re.search(pattern, content_lower)
        if match:
            if num_groups == 1:
                time_str = match.group(1)
                if len(time_str) == 4:
                    hour, minute = int(time_str[:2]), int(time_str[2:])
                    if 0 <= hour <= 23 and 0 <= minute <= 59:
                        extracted_time = f"{hour:02d}:{minute:02d}"
                        result['time_confidence'] = 'high'
                        break
            else:
                hour, minute = int(match.group(1)), int(match.group(2))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    extracted_time = f"{hour:02d}:{minute:02d}"
                    result['time_confidence'] = 'high'
                    break
    
    # HIGH confidence: Standard time formats without "hrs"
    if not extracted_time:
        time_patterns = [
            (r'at around (\d{1,2}):(\d{2})', 2), (r'at (\d{1,2}):(\d{2})', 2),
            (r'around (\d{1,2}):(\d{2})', 2),
            (r'at around (\d{1,2})\.(\d{2})', 2), (r'at (\d{1,2})\.(\d{2})', 2),
        ]
        for pattern, _ in time_patterns:
            match = re.search(pattern, content_lower)
            if match:
                hour, minute = int(match.group(1)), int(match.group(2))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    extracted_time = f"{hour:02d}:{minute:02d}"
                    result['time_confidence'] = 'high'
                    break
    
    # MEDIUM-HIGH confidence: Specific time markers
    if not extracted_time:
        time_markers = [
            (r'\bmidnight\b', '00:00', 'high'), (r'\bnoon\b|\bmidday\b', '12:00', 'high'),
            (r'\bdawn\b|\bsunrise\b', '06:00', 'medium'), (r'\bdusk\b|\bsunset\b', '19:00', 'medium'),
        ]
        for pattern, time, conf in time_markers:
            if re.search(pattern, content_lower):
                extracted_time, result['time_confidence'] = time, conf
                break
    
    # MEDIUM confidence: Time ranges (calculate midpoint)
    if not extracted_time:
        match = re.search(r'between (\d{1,2})[:\.]?(\d{2})?\s*(?:and|&|-)\s*(\d{1,2})[:\.]?(\d{2})?', content_lower)
        if match:
            hour1 = int(match.group(1))
            min1 = int(match.group(2)) if match.group(2) else 0
            hour2 = int(match.group(3))
            min2 = int(match.group(4)) if match.group(4) else 0
            if 0 <= hour1 <= 23 and 0 <= hour2 <= 23:
                mid_minutes = ((hour1 * 60 + min1) + (hour2 * 60 + min2)) // 2
                extracted_time = f"{mid_minutes // 60:02d}:{mid_minutes % 60:02d}"
                result['time_confidence'] = 'medium'
    
    # MEDIUM confidence: General time periods
    if not extracted_time:
        time_periods = [
            (r'\bearly hours\b', '03:00'), (r'\blate hours\b|\blate at night\b', '23:00'),
            (r'\bearly morning\b', '06:00'), (r'\bmorning\b', '09:00'),
            (r'\bafternoon\b', '15:00'), (r'\bevening\b', '19:00'), (r'\bnight\b', '22:00'),
        ]
        for pattern, time in time_periods:
            if re.search(pattern, content_lower):
                extracted_time, result['time_confidence'] = time, 'medium'
                break
    
    # Date extraction
    accident_date = None
    explicit_patterns = [
        (r'on (\d{1,2}(?:st|nd|rd|th)?\s+(?:january|february|march|april|may|june|july|august|september|october|november|december))', '%d %B'),
        (r'on ((?:january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{1,2}(?:st|nd|rd|th)?)', '%B %d'),
    ]
    
    for pattern, date_format in explicit_patterns:
        match = re.search(pattern, content_lower)
        if match:
            try:
                date_str = re.sub(r'(st|nd|rd|th)', '', match.group(1))
                accident_date = pd.to_datetime(date_str, format=date_format).replace(year=published_dt.year)
                if accident_date > published_dt + timedelta(days=30):
                    accident_date = accident_date.replace(year=published_dt.year - 1)
                break
            except:
                pass
    
    if accident_date is None:
        if re.search(r'^\s*today[,\s]|^today at', content_lower):
            accident_date = published_dt
        elif re.search(r'^\s*yesterday[,\s]|^yesterday at', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif re.search(r'\bthis morning\b', content_lower):
            accident_date = published_dt
        elif re.search(r'\blast night\b', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif re.search(r'\blast evening\b', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif match := re.search(r'\blast (monday|tuesday|wednesday|thursday|friday|saturday|sunday)', content_lower):
            days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
            days_back = (published_dt.weekday() - days.index(match.group(1))) % 7 or 7
            accident_date = published_dt - timedelta(days=days_back)
        else:
            accident_date = published_dt
    
    # Combine date and time
    if accident_date is not None:
        result['accident_date'] = accident_date.date()
        if extracted_time:
            try:
                hour, minute = map(int, extracted_time.split(':'))
                result['accident_datetime'] = accident_date.replace(hour=hour, minute=minute, second=0)
                result['accident_time'] = extracted_time
            except:
                result['accident_datetime'] = accident_date
        else:
            result['accident_datetime'] = accident_date
    
    return result

print("✓ Extraction function loaded")

✓ Extraction function loaded


# 3. Extract Accident DateTime for All Records

In [37]:
print("Extracting accident datetime...")

# Apply extraction
extraction_results = police_releases_df.apply(
    lambda row: extract_accident_datetime_improved(row['content'], row['date_published']), axis=1
)

# Add columns
extraction_df = pd.DataFrame(extraction_results.tolist())
for col in ['accident_datetime', 'accident_date', 'accident_time', 'time_confidence']:
    police_releases_df[col] = extraction_df[col]

# Add derived features
police_releases_df['accident_is_weekend'] = pd.to_datetime(police_releases_df['accident_datetime']).dt.dayofweek.isin([5, 6]).astype(int)
police_releases_df['publication_delay_hours'] = ((
    pd.to_datetime(police_releases_df['date_published']) - 
    pd.to_datetime(police_releases_df['accident_datetime'])
).dt.total_seconds() / 3600).apply(lambda x: max(0, x) if pd.notna(x) else x)

def categorize_time_of_day(hour):
    if pd.isna(hour): return None
    if 0 <= hour < 6: return 'Night (00:00-06:00)'
    if 6 <= hour < 12: return 'Morning (06:00-12:00)'
    if 12 <= hour < 18: return 'Afternoon (12:00-18:00)'
    return 'Evening (18:00-00:00)'

police_releases_df['time_of_day_category'] = pd.to_datetime(police_releases_df['accident_datetime']).dt.hour.apply(categorize_time_of_day)

# Add holiday/event/school indicators
def get_holiday_event_status(dt):
    """
    Determine if date is a holiday, event, or school holiday period.
    
    Returns: (is_holiday, is_event, is_school_holiday)
    """
    if pd.isna(dt):
        return 'no', 'no', 'no'
    
    date = dt.date() if hasattr(dt, 'date') else dt
    year = date.year
    month = date.month
    day = date.day
    
    # Maltese public holidays (fixed dates)
    # These are official public holidays that may affect traffic patterns due to
    # increased leisure travel, celebrations, and reduced commercial activity
    holidays = [
        (1, 1),   # New Year's Day
        (2, 10),  # St Paul's Shipwreck (Feast of St Paul's Shipwreck)
        (3, 19),  # St Joseph's Day
        (3, 31),  # Freedom Day (Jum il-Ħelsien)
        (5, 1),   # Workers' Day (May Day)
        (6, 7),   # Sette Giugno (Commemoration of 1919 riots)
        (6, 29),  # St Peter & St Paul (L-Imnarja - major feast)
        (8, 15),  # Assumption of Mary (Santa Marija - mid-summer holiday)
        (9, 8),   # Victory Day (Our Lady of Victories)
        (9, 21),  # Independence Day
        (12, 8),  # Immaculate Conception
        (12, 13), # Republic Day
        (12, 25), # Christmas Day
        (12, 26), # Boxing Day
    ]
    
    # Variable holidays for 2024-2025 (change yearly based on lunar calendar)
    # Good Friday and Easter Sunday dates vary each year
    variable_holidays = {
        2024: [(3, 29), (3, 30)],  # Good Friday, Easter Sunday 2024
        2025: [(4, 18), (4, 19)],  # Good Friday, Easter Sunday 2025
    }
    
    # School holidays periods (approximate)
    # School holidays may affect traffic patterns due to families traveling,
    # reduced rush-hour congestion, and increased daytime leisure traffic
    # Returns 'yes' if accident occurred during school holidays, 'no' otherwise
    school_holidays = [
        ('summer', 6, 20, 9, 15),    # Summer break: June 20 - Sept 15 (longest holiday)
        ('christmas', 12, 20, 1, 7), # Christmas break: Dec 20 - Jan 7 (crosses year boundary)
        ('easter', 4, 10, 4, 20),    # Easter break: April 10-20 (approx, varies with Easter date)
    ]
    
    # Notable events that affect traffic patterns (festa season, carnival, etc.)
    # These events involve road closures, processions, increased pedestrian traffic,
    # and visitors traveling to/from villages. Returns 'yes' during event periods
    events = {
        2024: [
            (2, 10, 2, 13),  # Carnival 2024 (weekend of celebrations, street parties)
            (6, 1, 9, 30),   # Festa season: June-Sept (village feasts throughout Malta)
        ],
        2025: [
            (3, 1, 3, 4),    # Carnival 2025 (weekend of celebrations, street parties)
            (6, 1, 9, 30),   # Festa season: June-Sept (village feasts throughout Malta)
        ]
    }
    
    is_holiday = 'no'
    is_event = 'no'
    is_school = 'no'
    
    # Check if it's a public holiday
    # Priority: 'eve of' overrides 'yes' if accident is day before a holiday
    if (month, day) in holidays:
        is_holiday = 'yes'
    elif year in variable_holidays and (month, day) in variable_holidays[year]:
        is_holiday = 'yes'
    
    # Check if it's eve of a holiday (day before)
    # Eve of holidays often have increased social activity and traffic
    next_day = date + timedelta(days=1)
    if (next_day.month, next_day.day) in holidays:
        is_holiday = 'eve of'
    elif year in variable_holidays and (next_day.month, next_day.day) in variable_holidays[year]:
        is_holiday = 'eve of'
    
    # Check if during event period
    if year in events:
        for event_start_m, event_start_d, event_end_m, event_end_d in events[year]:
            event_start = date.replace(month=event_start_m, day=event_start_d)
            event_end = date.replace(month=event_end_m, day=event_end_d)
            if event_start <= date <= event_end:
                is_event = 'yes'
                break
    
    # Check if during school holidays
    for holiday_name, start_m, start_d, end_m, end_d in school_holidays:
        try:
            if start_m <= end_m:  # Same year period
                start_date = date.replace(month=start_m, day=start_d)
                end_date = date.replace(month=end_m, day=end_d)
                if start_date <= date <= end_date:
                    is_school = 'yes'
                    break
            else:  # Crosses year boundary (e.g., Christmas)
                start_date = date.replace(month=start_m, day=start_d)
                end_date = date.replace(year=date.year+1 if date.month < 6 else date.year, month=end_m, day=end_d)
                if date >= start_date or date <= end_date:
                    is_school = 'yes'
                    break
        except:
            pass
    
    return is_holiday, is_event, is_school

# Apply holiday/event/school detection
police_releases_df[['is_holiday', 'is_event', 'is_school_holiday']] = police_releases_df['accident_datetime'].apply(
    lambda dt: pd.Series(get_holiday_event_status(dt))
)

# Summary
total = len(police_releases_df)
high = (police_releases_df['time_confidence'] == 'high').sum()
medium = (police_releases_df['time_confidence'] == 'medium').sum()
low = (police_releases_df['time_confidence'] == 'low').sum()

print(f"\n✓ Extraction complete!")
print(f"Total: {total} | High: {high} ({high/total*100:.1f}%) | Medium: {medium} ({medium/total*100:.1f}%) | Low: {low} ({low/total*100:.1f}%)")

Extracting accident datetime...

✓ Extraction complete!
Total: 111 | High: 106 (95.5%) | Medium: 1 (0.9%) | Low: 4 (3.6%)

✓ Extraction complete!
Total: 111 | High: 106 (95.5%) | Medium: 1 (0.9%) | Low: 4 (3.6%)


# 4. Manual Corrections for Low Confidence Records

In [38]:
# Display low confidence records
low_conf = police_releases_df[police_releases_df['time_confidence'] == 'low']
print(f"Low confidence records: {len(low_conf)}\n")

if len(low_conf) > 0:
    for idx, row in low_conf.head(10).iterrows():
        print(f"ID: {row['release_id']} | Published: {row['date_published']} | Current: {row['accident_time']}")
        print(f"Content: {row['content'][:150]}...")
        print("-" * 80)

Low confidence records: 4

ID: 16 | Published: 2025-04-17 | Current: None
Content: An 80-year-old man, a resident of Santa Luċija, was taken to Mater Dei Hospital for some injuries he sustained in a traffic accident in Luqa Road, Luq...
--------------------------------------------------------------------------------
ID: 25 | Published: 2024-12-27 | Current: None
Content: Today, at about 0745hrs, the Police were informed of a traffic accident in Triq Diċembru Tlettax, Marsa.The Police went immediately on site and prelim...
--------------------------------------------------------------------------------
ID: 76 | Published: 2025-04-14 | Current: None
Content: Yesterday, at about 1745hrs, the Police were informed of a traffic accident in Birżebbuġa Road, Birżebbuġa.Preliminary investigations found that a col...
--------------------------------------------------------------------------------
ID: 77 | Published: 2025-07-24 | Current: None
Content: A 65-year-old man, a resident of Mellieħa, w

In [39]:
# Manual corrections dictionary - add your corrections here
manual_corrections = {
    25: {'time': '07:45'},
    76: {'time': '17:45'},
    77: {'time': '10:15'}
    # Add more: release_id: {'time': 'HH:MM', 'date': 'YYYY-MM-DD'}
}

def apply_manual_corrections(df, corrections):
    """Apply manual time/date corrections to the dataframe."""
    df = df.copy()
    corrections_applied = 0
    
    for release_id, correction_data in corrections.items():
        mask = df['release_id'] == release_id
        if not mask.any():
            print(f"⚠ Release ID {release_id} not found")
            continue
        
        idx = df[mask].index[0]
        
        # Determine date
        if 'date' in correction_data:
            new_date = pd.to_datetime(correction_data['date'])
        else:
            new_date = pd.to_datetime(df.loc[idx, 'accident_date'] if pd.notna(df.loc[idx, 'accident_date']) 
                                     else df.loc[idx, 'date_published'])
        
        # Apply time
        if 'time' in correction_data:
            try:
                hour, minute = map(int, correction_data['time'].split(':'))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    df.loc[idx, 'accident_time'] = f"{hour:02d}:{minute:02d}"
                    df.loc[idx, 'accident_datetime'] = new_date.replace(hour=hour, minute=minute, second=0)
                    df.loc[idx, 'accident_date'] = new_date.date()
                    df.loc[idx, 'time_of_day_category'] = categorize_time_of_day(hour)
                    df.loc[idx, 'time_confidence'] = 'manual'
                    delay_hours = (pd.to_datetime(df.loc[idx, 'date_published']) - df.loc[idx, 'accident_datetime']).total_seconds() / 3600
                    df.loc[idx, 'publication_delay_hours'] = max(0, delay_hours)
                    corrections_applied += 1
                    print(f"✓ Release ID {release_id}: {correction_data['time']}")
                else:
                    print(f"✗ Invalid time for {release_id}: {correction_data['time']}")
            except Exception as e:
                print(f"✗ Error for {release_id}: {e}")
    
    return df, corrections_applied

if manual_corrections:
    police_releases_df, num_applied = apply_manual_corrections(police_releases_df, manual_corrections)
    print(f"\n✓ {num_applied} manual corrections applied")
else:
    print("No manual corrections defined")

✓ Release ID 25: 07:45
✓ Release ID 76: 17:45
✓ Release ID 77: 10:15

✓ 3 manual corrections applied


# 5. Final Statistics & Summary

In [40]:
print("=" * 80)
print("FINAL DATASET STATISTICS")
print("=" * 80)

total = len(police_releases_df)
high = (police_releases_df['time_confidence'] == 'high').sum()
medium = (police_releases_df['time_confidence'] == 'medium').sum()
manual = (police_releases_df['time_confidence'] == 'manual').sum()
low = (police_releases_df['time_confidence'] == 'low').sum()

print(f"\n📊 Confidence Distribution:")
print(f"   High:   {high:4d} ({high/total*100:5.1f}%)")
print(f"   Medium: {medium:4d} ({medium/total*100:5.1f}%)")
print(f"   Manual: {manual:4d} ({manual/total*100:5.1f}%)")
print(f"   Low:    {low:4d} ({low/total*100:5.1f}%)")
print(f"\n   Coverage: {high+medium+manual:4d} / {total} ({(high+medium+manual)/total*100:.1f}%)")

print(f"\n⏰ Time Distribution:")
print(police_releases_df['time_of_day_category'].value_counts().sort_index())

print(f"\n📅 Weekend vs Weekday:")
weekend_counts = police_releases_df['accident_is_weekend'].value_counts()
print(f"   Weekday: {weekend_counts.get(0, 0)}")
print(f"   Weekend: {weekend_counts.get(1, 0)}")

print(f"\n📰 Publication Timing:")
print(f"   Average delay: {police_releases_df['publication_delay_hours'].mean():.2f} hours")
print(f"   Median delay:  {police_releases_df['publication_delay_hours'].median():.2f} hours")

print(f"\n🎉 Holiday/Event/School Breakdown:")
print(f"   Holiday: {dict(police_releases_df['is_holiday'].value_counts())}")
print(f"   Event: {dict(police_releases_df['is_event'].value_counts())}")
print(f"   School holiday: {dict(police_releases_df['is_school_holiday'].value_counts())}")

print("\n" + "=" * 80)

# Sample output
sample_cols = ['release_id', 'accident_datetime', 'accident_time', 'accident_is_weekend', 
               'time_of_day_category', 'time_confidence', 'is_holiday', 'is_event', 'is_school_holiday']
               
print(police_releases_df[sample_cols].head(10))

FINAL DATASET STATISTICS

📊 Confidence Distribution:
   High:    106 ( 95.5%)
   Medium:    1 (  0.9%)
   Manual:    3 (  2.7%)
   Low:       1 (  0.9%)

   Coverage:  110 / 111 (99.1%)

⏰ Time Distribution:
time_of_day_category
Afternoon (12:00-18:00)    29
Evening (18:00-00:00)      27
Morning (06:00-12:00)      46
Night (00:00-06:00)         9
Name: count, dtype: int64

📅 Weekend vs Weekday:
   Weekday: 74
   Weekend: 37

📰 Publication Timing:
   Average delay: 1.55 hours
   Median delay:  0.00 hours

🎉 Holiday/Event/School Breakdown:
   Holiday: {'no': np.int64(102), 'eve of': np.int64(7), 'yes': np.int64(2)}
   Event: {'no': np.int64(69), 'yes': np.int64(42)}
   School holiday: {'yes': np.int64(85), 'no': np.int64(26)}

   release_id   accident_datetime accident_time  accident_is_weekend  \
0           1 2025-10-09 09:30:00         09:30                    0   
1           2 2025-06-19 18:30:00         18:30                    0   
2           3 2025-05-12 08:00:00         08:00  

# 6. Save Final Datasets

In [42]:
# Save full dataset
police_releases_df.to_csv(f"{OUTPUT_PATH}police_releases_with_datetime.csv", index=False)

# # Save summary dataset
# summary_columns = [
#     'release_id', 'date_published', 'accident_datetime', 'accident_date', 
#     'accident_time', 'accident_hour', 'time_of_day_category',
#     'accident_day_of_week', 'accident_is_weekend', 'time_confidence',
#     'publication_delay_hours', 'is_holiday', 'is_event', 'is_school_holiday'
# ]
# police_releases_df[summary_columns].to_csv(f"{OUTPUT_PATH}police_releases_summary.csv", index=False)

# # Export low confidence for external review (CSV)
# low_conf = police_releases_df[police_releases_df['time_confidence'] == 'low'][
#     ['release_id', 'date_published', 'accident_time', 'accident_date', 'content']
# ].copy()
# low_conf['corrected_time'] = ''
# low_conf['corrected_date'] = ''
# low_conf.to_csv(f"{OUTPUT_PATH}low_confidence_review.csv", index=False)

print("✓ Datasets saved successfully!\n")
print("Files created:")
print(f"1. {OUTPUT_PATH}police_releases_with_datetime.csv - Full dataset")
# print(f"2. {OUTPUT_PATH}police_releases_summary.csv - Summary with key features")
# print(f"3. {OUTPUT_PATH}low_confidence_review.csv - Low confidence records for manual review")
print(f"\n📊 Dataset ready for analysis and modeling!")

✓ Datasets saved successfully!

Files created:
1. C:\Python\ics5110-assignment\data\processed\police_releases_with_datetime.csv - Full dataset

📊 Dataset ready for analysis and modeling!
